In [4]:
import torch
from torch import nn
from d2l import torch as d2l

In [12]:
def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    if not torch.is_grad_enabled(): #如果是预测模式，这为False
        X_hat=(X-moving_mean)/torch.sqrt(moving_var+eps)
    else:
        #分来那种情况讨论，如果X是全连接层，len(X)==2 ,如果作用于卷积层在非线性函数之前，这len(X)==4;
        assert len(X) in (2,4)
        if len(X)==2:
            mean=torch.mean(X,dim=0,keepdim=True)
            var=((X-mean)**2).mean(dim=0)
            
        else:
            mean=torch.mean(X,dim=(0,2,3),keepdim=True)
            var=((X-mean)**2).mean(dim=(0,2,3),keepdim=True)
        
        X_hat=(X-mean)/torch.sqrt(var+eps)
             # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 缩放和移位
    return Y, moving_mean.data, moving_var.data

In [13]:
class BatchNorm(nn.Module):
    def __init__(self,numfeature,numdim):
        super().__init__()
        if(numdim==2): #定义Gammma，eps,Moving Mean,Moving variance
            shape=(1,numfeature)
        else:
            shape=(1,numfeature,1,1)
        self.gamma=nn.parameter(torch.ones(shape))
        self.beta=nn.parameter(torch.zeros(shape))
        self.moving_mean=torch.zeros(shape)
        self.moving_var=torch.ones(shape)
    def forward(self,X):
        if self.moving_mean.device!=X.device:
            self.moving_mean.device=self.moving_mean.to(X.device)
            self.moving_var.device=self.moving_var.to(X.device)
        
        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y


In [52]:
#Residual 
from torch.nn import functional as F
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1) -> None:
        super().__init__()
        self.conv1=nn.Conv2d(input_channels,num_channels,kernel_size=3,stride=strides,padding=1)
        self.conv2=nn.Conv2d(num_channels,num_channels,kernel_size=3,padding=1)

        if use_1x1conv:
            self.conv3=nn.Conv2d(input_channels,num_channels,kernel_size=1,stride=strides)
        else:
            self.conv3=None
        
        self.norm1=nn.BatchNorm2d(num_channels)
        self.norm2=nn.BatchNorm2d(num_channels)
    def forward(self,X):
        F1=F.relu(self.norm1(self.conv1(X)))
        F2=self.norm2(self.conv2(F1))
        if self.conv3:
            X=self.conv3(X)
        F2+=X
        return F.relu(F2)



In [53]:
#定义一个ResNet
b1=nn.Sequential(nn.Conv2d(1,64,kernel_size=7,padding=3,stride=2),nn.BatchNorm2d(64),nn.ReLU(),nn.MaxPool2d(kernel_size=3,stride=2,padding=1))

In [54]:
def resnet_block(input_channels,num_channels,num_residuals,isFirstBlock):
    block=[]
    for i in range(num_residuals):
        if i==0 and not isFirstBlock:
            block.append(Residual(input_channels,num_channels,True,strides=2))
        else:
            block.append(Residual(num_channels,num_channels))
    return block

In [55]:
net=nn.Sequential(b1,nn.Sequential(*resnet_block(64,64,2,isFirstBlock=True)),
                  nn.Sequential(*resnet_block(64,128,2,isFirstBlock=False)),
                  nn.Sequential(*resnet_block(128,256,2,isFirstBlock=False)),
                  nn.Sequential(*resnet_block(256,512,2,isFirstBlock=False)),
                  nn.AdaptiveAvgPool2d((1,1)),nn.Flatten(),nn.Linear(512,10))

In [56]:
X=torch.rand(size=(1,1,224,224))
for layer in net:
    X=layer(X)
    print(layer.__class__.__name__,"OutPutShape:",X.shape)

Sequential OutPutShape: torch.Size([1, 64, 56, 56])
Sequential OutPutShape: torch.Size([1, 64, 56, 56])
Sequential OutPutShape: torch.Size([1, 128, 28, 28])
Sequential OutPutShape: torch.Size([1, 256, 14, 14])
Sequential OutPutShape: torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d OutPutShape: torch.Size([1, 512, 1, 1])
Flatten OutPutShape: torch.Size([1, 512])
Linear OutPutShape: torch.Size([1, 10])


In [ ]:
lr, num_epochs, batch_size = 0.05, 10, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

DenseNet